In [1]:
from general_trader import GeneralTrader
import polars as pl
import polars_talib as pltalib
from trading_helpers import TradingData

In [2]:
traderData = TradingData(
    "/mnt/jonarne/dev/trading_data/",
    "/SP500/",
    "SP500_list.parquet",
)
# download_all_tickers. This function is only needed to call once,
# or when you decide to download new fresh ticker and sector data
#traderData.download_all_tickers()


# populate polars dataframe with open, close, high, low and volume data
ochl_data = traderData.load_ticker_parquet_files()
print(ochl_data.collect())


TradingData Initialized
shape: (4_300_704, 7)
┌─────────────────────┬───────────┬────────────┬───────────┬───────────┬─────────┬────────┐
│ date                ┆ close     ┆ high       ┆ low       ┆ open      ┆ volume  ┆ ticker │
│ ---                 ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---     ┆ ---    │
│ datetime[ns]        ┆ f64       ┆ f64        ┆ f64       ┆ f64       ┆ i64     ┆ str    │
╞═════════════════════╪═══════════╪════════════╪═══════════╪═══════════╪═════════╪════════╡
│ 1994-10-18 00:00:00 ┆ 1.205047  ┆ 1.214462   ┆ 1.186218  ┆ 1.205047  ┆ 418166  ┆ O      │
│ 1994-10-19 00:00:00 ┆ 1.308607  ┆ 1.318021   ┆ 1.214463  ┆ 1.214463  ┆ 249950  ┆ O      │
│ 1994-10-20 00:00:00 ┆ 1.318019  ┆ 1.365091   ┆ 1.318019  ┆ 1.327434  ┆ 141590  ┆ O      │
│ 1994-10-21 00:00:00 ┆ 1.299191  ┆ 1.327434   ┆ 1.289776  ┆ 1.318019  ┆ 68731   ┆ O      │
│ 1994-10-24 00:00:00 ┆ 1.270948  ┆ 1.327435   ┆ 1.270948  ┆ 1.299192  ┆ 75749   ┆ O      │
│ …                   ┆ …         

In [3]:


    # Define a sample Polars expression
    #my_filter = (pl.col("close") > 100) & (pl.col("volume") > 50000)
my_filter = None
    # --- Test Cases ---

print("--- Testing Valid Instantiation ---")
try:
    trader = GeneralTrader(ohlcColumnsToUse=["close"], df_ohlc=ochl_data.collect(), initialFilter=my_filter)
    print("Successfully created GeneralTrader instance.")
    print("DataFrame Schema:", trader.df_ohlc.schema)
    print("Initial Filter:", trader.initialFilter)
except ValidationError as e:
    print("Validation Error (unexpected):", e)

--- Testing Valid Instantiation ---
Successfully created GeneralTrader instance.
DataFrame Schema: Schema([('ticker', String), ('close', Float64), ('date', Datetime(time_unit='ns', time_zone=None))])
Initial Filter: None


In [4]:
trader.df_ohlc

ticker,close,date
str,f64,datetime[ns]
"""O""",1.205047,1994-10-18 00:00:00
"""O""",1.308607,1994-10-19 00:00:00
"""O""",1.318019,1994-10-20 00:00:00
"""O""",1.299191,1994-10-21 00:00:00
"""O""",1.270948,1994-10-24 00:00:00
…,…,…
"""NRG""",94.470001,2025-04-11 00:00:00
"""NRG""",95.919998,2025-04-14 00:00:00
"""NRG""",97.110001,2025-04-15 00:00:00


In [5]:
indicators = {
    "sma": pltalib.sma(timeperiod=200).over("ticker"),
    "mean_B": pl.col("close").mean().over("ticker"),
    #"bad_stuff": 5,
    "A_plus_B": pl.col("close") + pl.col("close"),
}

filterExpr =  pl.col("A_plus_B") > 100
trader.add_columns_and_prefilter(indicators, filterExpr)

GeneralTrader(df_ohlc=shape: (35_964, 6)
┌────────┬────────────┬─────────────────────┬────────────┬────────────┬─────────────┐
│ ticker ┆ close      ┆ date                ┆ sma        ┆ mean_B     ┆ A_plus_B    │
│ ---    ┆ ---        ┆ ---                 ┆ ---        ┆ ---        ┆ ---         │
│ str    ┆ f64        ┆ datetime[ns]        ┆ f64        ┆ f64        ┆ f64         │
╞════════╪════════════╪═════════════════════╪════════════╪════════════╪═════════════╡
│ MPWR   ┆ 503.812164 ┆ 2021-09-23 00:00:00 ┆ 372.915482 ┆ 151.423233 ┆ 1007.624329 │
│ MPWR   ┆ 503.977631 ┆ 2021-09-24 00:00:00 ┆ 373.868979 ┆ 151.423233 ┆ 1007.955261 │
│ MPWR   ┆ 504.186035 ┆ 2021-10-21 00:00:00 ┆ 386.74126  ┆ 151.423233 ┆ 1008.37207  │
│ MPWR   ┆ 509.359344 ┆ 2021-10-25 00:00:00 ┆ 388.049812 ┆ 151.423233 ┆ 1018.718689 │
│ MPWR   ┆ 509.856262 ┆ 2021-10-26 00:00:00 ┆ 388.696037 ┆ 151.423233 ┆ 1019.712524 │
│ …      ┆ …          ┆ …                   ┆ …          ┆ …          ┆ …           │
│ ERIE   ┆ 52

In [6]:
trader.df_ohlc

ticker,close,date,sma,mean_B,A_plus_B
str,f64,datetime[ns],f64,f64,f64
"""MPWR""",503.812164,2021-09-23 00:00:00,372.915482,151.423233,1007.624329
"""MPWR""",503.977631,2021-09-24 00:00:00,373.868979,151.423233,1007.955261
"""MPWR""",504.186035,2021-10-21 00:00:00,386.74126,151.423233,1008.37207
"""MPWR""",509.359344,2021-10-25 00:00:00,388.049812,151.423233,1018.718689
"""MPWR""",509.856262,2021-10-26 00:00:00,388.696037,151.423233,1019.712524
…,…,…,…,…,…
"""ERIE""",525.870056,2024-10-10 00:00:00,400.609192,81.766443,1051.740112
"""ERIE""",531.064514,2024-10-11 00:00:00,401.642105,81.766443,1062.129028
"""ERIE""",529.644226,2024-10-14 00:00:00,402.663161,81.766443,1059.288452
